# Morbilidad Hospitalaria España 2016-2019
[Datos INE](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176778&menu=resultados&secc=1254736195291&idp=1254735573175#!tabs-1254736195291)

In [1]:
import pandas as pd
import os
import datetime

In [2]:
os.listdir('../data/morbilidad')

['anterior',
 'codigos.xlsx',
 'EMH_2016_G01.csv',
 'EMH_2017_G01.csv',
 'EMH_2018_G01.csv',
 'EMH_2019_G01.csv']

In [3]:
morbi16 = pd.read_csv('../data/morbilidad/EMH_2016_G01.csv',sep='\t')
morbi17 = pd.read_csv('../data/morbilidad/EMH_2017_G01.csv',sep='\t')
morbi18 = pd.read_csv('../data/morbilidad/EMH_2018_G01.csv',sep='\t')
morbi19 = pd.read_csv('../data/morbilidad/EMH_2019_G01.csv',sep='\t')

In [4]:
morbi16.sample(5)

,Norden,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,EdadMes,EdadDias,DiasEstancia,Factor
539739,2556730,3,2,3,2,30082016,D500,1,69,6,2,4,1.00000
197866,7475854,8,1,8,1,6052016,C320,1,67,1,7,1,1.49795
864925,13340850,11,1,11,2,12122016,G893,1,66,10,13,4,1.00000
683363,818539,3,1,3,2,16092016,F15188,1,46,0,23,3,1.00000
127737,9401215,8,1,8,1,11102016,C180,1,75,0,23,6,1.01123


In [5]:
morbilidad = pd.concat([morbi16,morbi17,morbi18,morbi19])

In [6]:
morbilidad.sample(5)

,Norden,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,EdadMes,EdadDias,DiasEstancia,Factor
738343,44651150,48,2,48,1,15062017,F259,1,55,3,26,28,1.00000
503941,24314355,28,2,28,1,2032018,D252,1,49,4,1,3,1.01195
588645,44747394,20,2,20,2,9022017,D701,1,5,2,20,7,1.00000
315327,19713528,24,1,24,1,26012017,C61,1,61,6,22,4,1.00000
374228,39333020,43,1,43,1,31082017,C679,1,79,7,19,2,1.00000


In [7]:
morbilidad = morbilidad[morbilidad["DiagPrin"].str.contains('^F')]

In [8]:
morbilidad.DiagPrin = morbilidad.DiagPrin.str.strip()

In [9]:
icd10 = pd.read_csv('../data/icd10.csv')

In [10]:
icd10_psic = icd10[icd10["code"].str.contains('^F')]

In [11]:
icd10_psic.sample(3)

,code,diag
4546,F4521,Hypochondriasis
4543,F450,Somatization disorder
4217,F1520,"Other stimulant dependence, uncomplicated"


In [12]:
morbilidad.drop(['Norden','EdadDias','EdadMes'],axis=1,inplace = True)

In [13]:
morbilidad.DiagPrin.unique()[0:20] #extraer F + 2 primero dígitos

array(['F0150', 'F0151', 'F0390', 'F0391', 'F04', 'F05', 'F060', 'F061',
       'F062', 'F0630', 'F0631', 'F0632', 'F0633', 'F0634', 'F064',
       'F068', 'F070', 'F0781', 'F0789', 'F079'], dtype=object)

In [14]:
morbilidad["DiagGen"] = morbilidad.DiagPrin.str.extract(r'(^F..)')

In [15]:
morbilidad.DiagGen.unique()

array(['F01', 'F03', 'F04', 'F05', 'F06', 'F07', 'F09', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29', 'F30', 'F31',
       'F32', 'F33', 'F34', 'F39', 'F40', 'F41', 'F42', 'F43', 'F44',
       'F45', 'F48', 'F50', 'F51', 'F52', 'F53', 'F55', 'F59', 'F60',
       'F63', 'F64', 'F65', 'F66', 'F68', 'F69', 'F70', 'F71', 'F72',
       'F73', 'F78', 'F79', 'F80', 'F81', 'F82', 'F84', 'F88', 'F89',
       'F90', 'F91', 'F93', 'F94', 'F95', 'F98', 'F99'], dtype=object)

In [16]:
diagnos = pd.read_csv("../data/diagnos.csv")

In [17]:
diagnos.sample(5)

,code,diag
8,F80-89,Trastornos del desarrollo psicológico
58,F59,Síndromes de comportamiento sin especificar as...
3,F30-39,"Trastornos del estado de animo, trastornos afe..."
85,F98,Otros trastornos emocionales y de comportamien...
62,F64,Trastornos de identidad de género


In [18]:
dicc_diag = {} #crear diccionario con diagnósticos y códigos
for i,r in diagnos.iterrows():
    dicc_diag[r.code] = r.diag

In [19]:
morbilidad["FFF"] = morbilidad.DiagGen.replace(dicc_diag) #columna con subclasificación diagnoósticos.

In [20]:
F0 = ["F00","F01","F02","F03","F04","F05","F06","F07","F08","F09"]
F1 = ["F10","F11","F12","F13","F14","F15","F16","F17","F18","F19"]
F2 = ["F20","F21","F22","F23","F24","F25","F26","F27","F28","F29"]
F3 = ["F30","F31","F32","F33","F34","F35","F36","F37","F38","F39"]
F4 = ["F40","F41","F42","F43","F44","F45","F46","F47","F48","F49"]
F5 = ["F50","F51","F52","F53","F54","F55","F56","F57","F58","F59"]
F6 = ["F60","F61","F62","F63","F64","F65","F66","F67","F68","F69"]
F7 = ["F70","F71","F72","F73","F74","F75","F76","F77","F78","F79"]
F8 = ["F80","F81","F82","F83","F84","F85","F86","F87","F88","F89"]
F9 = ["F90","F91","F92","F93","F94","F95","F96","F97","F98"]
F99 = "F99"

In [21]:
dicc_f = {}

In [22]:
cabezera = {'F00-F09': 'Trastornos mentales orgánicos, incluidos los trastornos sintomáticos',
 'F10-F19': 'Trastornos mentales y de comportamiento debidos al consumo de psicotrópicos',
 'F20-29': 'Esquizofrenia, trastornos esquizotípicos y trastornos delirantes',
 'F30-39': 'Trastornos del estado de animo, trastornos afectivos',
 'F40-49': 'Trastornos neuróticos, trastornos relacionados con el estrés y trastornos somatomorfos',
 'F50-59': 'Síndromes del comportamiento asociados con alteraciones fisiológicas y factores físicos',
 'F60-69': 'Trastornos de la personalidad y del comportamiento en adultos',
 'F70-79': 'Retraso mental',
 'F80-89': 'Trastornos del desarrollo psicológico',
 'F90-F98': 'Trastornos emocionales y del comportamiento que aparecen habitualmente en la niñez o en la adolescencia'}

In [23]:
capi = diagnos.head(11)

In [24]:
capi

,code,diag
0,F00-F09,"Trastornos mentales orgánicos, incluidos los t..."
1,F10-F19,Trastornos mentales y de comportamiento debido...
2,F20-29,"Esquizofrenia, trastornos esquizotípicos y tra..."
3,F30-39,"Trastornos del estado de animo, trastornos afe..."
4,F40-49,"Trastornos neuróticos, trastornos relacionados..."
5,F50-59,Síndromes del comportamiento asociados con alt...
6,F60-69,Trastornos de la personalidad y del comportami...
7,F70-79,Retraso mental
8,F80-89,Trastornos del desarrollo psicológico
9,F90-F98,Trastornos emocionales y del comportamiento qu...


In [25]:
cabezera = {}
for i,r in capi.iterrows():
    cabezera[r.code] = r.diag
cabezera

{'F00-F09': 'Trastornos mentales orgánicos, incluidos los trastornos sintomáticos',
 'F10-F19': 'Trastornos mentales y de comportamiento debidos al consumo de psicotrópicos',
 'F20-29': 'Esquizofrenia, trastornos esquizotípicos y trastornos delirantes',
 'F30-39': 'Trastornos del estado de animo, trastornos afectivos',
 'F40-49': 'Trastornos neuróticos, trastornos relacionados con el estrés y trastornos somatomorfos',
 'F50-59': 'Síndromes del comportamiento asociados con alteraciones fisiológicas y factores físicos',
 'F60-69': 'Trastornos de la personalidad y del comportamiento en adultos',
 'F70-79': 'Retraso mental',
 'F80-89': 'Trastornos del desarrollo psicológico',
 'F90-F98': 'Trastornos emocionales y del comportamiento que aparecen habitualmente en la niñez o en la adolescencia',
 'F99': 'Trastornos mentales sin especificar'}

In [26]:
capitulando = {}
for f in F0:
    capitulando[f] = cabezera['F00-F09']
for f in F1:
    capitulando[f] = cabezera['F10-F19']
for f in F2:
    capitulando[f] = cabezera['F20-29']
for f in F3:
    capitulando[f] = cabezera['F30-39']
for f in F4:
    capitulando[f] = cabezera['F40-49']
for f in F5:
    capitulando[f] = cabezera['F50-59']
for f in F6:
    capitulando[f] = cabezera['F60-69']
for f in F7:
    capitulando[f] = cabezera['F70-79']
for f in F8:
    capitulando[f] = cabezera['F80-89']
for f in F9:
    capitulando[f] = cabezera['F90-F98']
capitulando[F99] = cabezera['F99']
#capitulando
    

In [27]:
morbilidad.sample(5)

,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,DiagGen,FFF
727924,29,2,29,2,13062017,F22,1,42,5,1.04211,F22,Trastornos delirantes persistentes
745726,46,1,46,2,15052018,F312,1,36,45,1.00000,F31,Trastorno bipolar
732236,3,2,3,2,1042016,F329,1,51,43,1.00000,F32,"Trastorno depresivo mayor, episodio único"
735611,28,2,28,2,31082016,F331,1,76,17,1.08548,F33,"Trastorno depresivo mayor, recurrente"
730960,48,2,48,2,19052016,F322,1,72,24,1.00441,F32,"Trastorno depresivo mayor, episodio único"


In [28]:
morbilidad["Fgen"] = morbilidad.DiagGen.replace(capitulando)

In [29]:
discapacidad = morbilidad[morbilidad["Fgen"] == 'Retraso mental'].index

In [30]:
morbilidad.drop(discapacidad,inplace = True)

In [31]:
morbilidad.to_csv("../data/morbilidad_Fgen.csv",index=False)

In [32]:
morbilidad.Fgen.unique()

array(['Trastornos mentales orgánicos, incluidos los trastornos sintomáticos',
       'Trastornos mentales y de comportamiento debidos al consumo de psicotrópicos',
       'Esquizofrenia, trastornos esquizotípicos y trastornos delirantes',
       'Trastornos del estado de animo, trastornos afectivos',
       'Trastornos neuróticos, trastornos relacionados con el estrés y trastornos somatomorfos',
       'Síndromes del comportamiento asociados con alteraciones fisiológicas y factores físicos',
       'Trastornos de la personalidad y del comportamiento en adultos',
       'Trastornos del desarrollo psicológico',
       'Trastornos emocionales y del comportamiento que aparecen habitualmente en la niñez o en la adolescencia',
       'Trastornos mentales sin especificar'], dtype=object)

In [33]:
morbilidad.sample(5)

,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,DiagGen,FFF,Fgen
767486,8,1,8,1,12082016,F919,2,17,45,1.02915,F91,Trastornos de conducta,Trastornos emocionales y del comportamiento qu...
684990,46,1,46,2,19092016,F19950,1,32,7,1.00000,F19,Trastornos mentales y de comportamiento debido...,Trastornos mentales y de comportamiento debido...
763499,28,1,28,2,3032017,F332,1,44,4,1.00000,F33,"Trastorno depresivo mayor, recurrente","Trastornos del estado de animo, trastornos afe..."
773714,28,2,28,2,29012019,F322,1,53,48,1.01201,F32,"Trastorno depresivo mayor, episodio único","Trastornos del estado de animo, trastornos afe..."
731819,2,1,2,1,1092017,F250,1,24,4,1.00000,F25,Trastornos esquizoafectivos,"Esquizofrenia, trastornos esquizotípicos y tra..."


In [34]:
reverse_dic = {}
for i,r in capi.iterrows():
    reverse_dic[r.diag] = r.code
reverse_dic

{'Trastornos mentales orgánicos, incluidos los trastornos sintomáticos': 'F00-F09',
 'Trastornos mentales y de comportamiento debidos al consumo de psicotrópicos': 'F10-F19',
 'Esquizofrenia, trastornos esquizotípicos y trastornos delirantes': 'F20-29',
 'Trastornos del estado de animo, trastornos afectivos': 'F30-39',
 'Trastornos neuróticos, trastornos relacionados con el estrés y trastornos somatomorfos': 'F40-49',
 'Síndromes del comportamiento asociados con alteraciones fisiológicas y factores físicos': 'F50-59',
 'Trastornos de la personalidad y del comportamiento en adultos': 'F60-69',
 'Retraso mental': 'F70-79',
 'Trastornos del desarrollo psicológico': 'F80-89',
 'Trastornos emocionales y del comportamiento que aparecen habitualmente en la niñez o en la adolescencia': 'F90-F98',
 'Trastornos mentales sin especificar': 'F99'}

In [35]:
morbilidad.FxAlta = morbilidad.FxAlta.astype('string')

In [36]:
morbilidad["CodGen"] = morbilidad.Fgen.replace(reverse_dic)

In [37]:
morbilidad['fecha_al'] = pd.to_datetime({'year': morbilidad.FxAlta.str[-4:],
                           'month': morbilidad.FxAlta.str[-6:-4],
                           'day': morbilidad.FxAlta.str[:-6]
                           })

In [38]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,DiagGen,FFF,Fgen,CodGen,fecha_al
768954,15,1,15,2,13072017,F411,1,75,1,1.00000,F41,Otros trastornos de ansiedad,"Trastornos neuróticos, trastornos relacionados...",F40-49,2017-07-13
733161,25,1,25,1,4072018,F259,1,67,50,4.37919,F25,Trastornos esquizoafectivos,"Esquizofrenia, trastornos esquizotípicos y tra...",F20-29,2018-07-04
721057,8,1,8,1,22062018,F209,1,22,35,1.26359,F20,Esquizofrenia,"Esquizofrenia, trastornos esquizotípicos y tra...",F20-29,2018-06-22


In [39]:
morbilidad['fecha_ing'] = morbilidad.apply(lambda fila: fila['fecha_al'] - datetime.timedelta(days = fila['DiasEstancia']), axis= 1)

In [40]:
#diccionarios códigos
dic_pro ={2:'Albacete',3:'Alicante',4:'Almería',1:'Álava',33:'Asturias',5:'Ávila',6:'Badajoz',7:'Balears, Illes',
8:'Barcelona',48:'Bizkaia',9:'Burgos',10:'Cáceres',11:'Cádiz',39:'Cantabria',12:'Castellón',13:'Ciudad Real',
14:'Córdoba',15:'Coruña, A',16:'Cuenca',20:'Gipuzkoa',17:'Girona',18:'Granada',19:'Guadalajara',21:'Huelva',22:'Huesca',
23:'Jaén',24:'León',25:'Lleida',27:'Lugo',28:'Madrid',29:'Málaga',30:'Murcia',31:'Navarra',32:'Ourense',34:'Palencia',
35:'Palmas, Las',36:'Pontevedra',26:'Rioja, La',37:'Salamanca',38:'Santa Cruz de Tenerife',40:'Segovia',41:'Sevilla',42:'Soria',
43:'Tarragona',44:'Teruel',45:'Toledo',46:'Valencia',47:'Valladolid',49:'Zamora',50:'Zaragoza',51:'Ceuta',
52:'Melilla',53:'Extranjeros'}
dic_sexo = {1:'Hombre',2:'Mujer'}
dic_entrada = {1:'Ordinario',2:'Urgente'}
dic_alta = {1: "Curación",2:'Traslado',3:'Fallecimiento',4:'Otras'}

In [41]:
morbilidad.ProvHosp = morbilidad.ProvHosp.map(dic_pro) #ProvResi
morbilidad.ProvResi = morbilidad.ProvResi.map(dic_pro)
morbilidad.DiagEntr = morbilidad.DiagEntr.map(dic_entrada)
morbilidad.MotivoAlta = morbilidad.MotivoAlta.map(dic_alta)
morbilidad.Sexo = morbilidad.Sexo.map(dic_sexo)

In [42]:
morbilidad.drop(['FxAlta','DiagPrin'],axis=1,inplace = True)

In [43]:
morbilidad.columns = ['ProvHosp', 'Sexo', 'ProvResi', 'Urgencia', 'MotivoAlta', 'Edad',
       'Dias', 'Factor', 'CodeDiag', 'Diag', 'DiagGen', 'CodeGen',
       'fecha_al', 'fecha_ing']

In [44]:
morbilidad = morbilidad.iloc[:,[1,5,0,2,3,12,13,6,4,8,9,11,10,7]]

In [45]:
morbilidad.Dias.max()

24517

In [46]:
morbilidad.sample(3)

,Sexo,Edad,ProvHosp,ProvResi,Urgencia,fecha_al,fecha_ing,Dias,MotivoAlta,CodeDiag,Diag,CodeGen,DiagGen,Factor
752223,Hombre,34,Alicante,Alicante,Urgente,2019-10-04,2019-09-29,5,Curación,F25,Trastornos esquizoafectivos,F20-29,"Esquizofrenia, trastornos esquizotípicos y tra...",1.00000
703609,Hombre,78,Santa Cruz de Tenerife,Santa Cruz de Tenerife,Urgente,2016-03-22,2016-03-07,15,Curación,F22,Trastornos delirantes persistentes,F20-29,"Esquizofrenia, trastornos esquizotípicos y tra...",1.00000
698485,Mujer,19,Barcelona,Barcelona,Urgente,2016-07-01,2016-06-17,14,Curación,F20,Esquizofrenia,F20-29,"Esquizofrenia, trastornos esquizotípicos y tra...",1.01123


In [47]:
morbilidad.to_csv("../data/morbilidad_Fgen.csv",index=False)